In [65]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from multiprocessing import Process
from multiprocessing import Pool
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [69]:
options = Options()

options.add_argument('--user-data-dir=/Users/Arian_Gh/AppData/Local/Google/Chrome/User Data/') 
options.add_argument('--profile-directory=Profile 6')
options.add_argument("--disable-extensions")
options.add_argument('--disable-blink-features=AutomationControlled')


options.add_argument('disable-infobars')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-debugging-port=9222")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

prefs = {
    "profile.default_content_setting_values.images": 2
}
options.add_experimental_option("prefs", prefs)

# Add a preference to disable image loading


capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True


driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)

In [70]:
driver.get("https://dir.indiamart.com/search.mp?ss=scrap&prdsrc=1&stype=attr=1|attrS&res=RC4&qu-comp=to")

In [71]:
file_path = 'categories.txt'

categories = []

with open(file_path, 'r') as file:
    for line in file:     
        category = line.strip()
        categories.append(category)

In [74]:

all_details = {}
index = 0
urls = []
soup_old = ""
while True:
    
    html_text = driver.page_source
    soup = BeautifulSoup(html_text,"lxml")
    
    products = soup.find("div",class_='right-group flx1 df fww pr')
    sections = products.find_all("section")

    for section in sections:
        details={}
        try:
            name = section.find("a",class_="prd-name").text
            price = section.find("span",class_="elps elps2 fs14 tac clr7").text
            url = section.find("a",class_="prd-name")["href"].split(" ")[0]
            if url in urls:
                continue

            details["name"]= name
            details["price"] = price
            details["url"] = url

            urls.append(url)
            all_details[index] = details
            index+=1
        except:
            try:
                name = section.find("span",class_="noPdp prd-name").text
                price = section.find("span",class_="elps elps2 fs14 tac clr7").text
                url = None
                details["name"]= name
                details["price"] = price
                details["url"] = url
                all_details[index] = details
                index+=1
            except:
                continue
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    if soup_old == soup:
        break
    else:
        print("yeki nistan motasefane XD")
        soup_old = soup
    try:
        btn = driver.find_element(By.XPATH, "//*[@id='app']/div/main/div[4]/div[2]/div[2]/span")

        driver.execute_script("arguments[0].click();",btn)
        
    except:
        break

IndentationError: expected an indented block after 'if' statement on line 25 (2405441947.py, line 26)

In [76]:
df = pd.DataFrame().from_dict(all_details).T